In [3]:
# ==============================
# ✅ Install Dependencies
# ==============================
!pip install -q gradio tensorflow matplotlib seaborn opencv-python

# ==============================
# 📦 Import Libraries
# ==============================
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import cv2
import random
from PIL import Image
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import gradio as gr

# ==============================
# 🧪 Set Seeds & Constants
# ==============================
np.random.seed(42)
tf.random.set_seed(42)

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10
NUM_CLASSES = 2
LEARNING_RATE = 0.0001

# ==============================
# 📁 Generate Synthetic Dataset
# ==============================
def create_sample_dataset():
    for category in ['train', 'validation', 'test']:
        for label in ['fresh', 'rotten']:
            os.makedirs(f'dataset/{category}/{label}', exist_ok=True)

    def create_sample_image(path, color, text):
        img = np.zeros((224, 224, 3), dtype=np.uint8)
        img[:, :, :] = color
        cv2.putText(img, text, (50, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)
        cv2.imwrite(path, img)

    for i in range(50):
        create_sample_image(f'dataset/train/fresh/fresh_{i}.jpg', [0, 150, 0], "FRESH")
        create_sample_image(f'dataset/train/rotten/rotten_{i}.jpg', [150, 0, 0], "ROTTEN")

    for i in range(15):
        create_sample_image(f'dataset/validation/fresh/fresh_{i}.jpg', [0, 150, 0], "FRESH")
        create_sample_image(f'dataset/validation/rotten/rotten_{i}.jpg', [150, 0, 0], "ROTTEN")
        create_sample_image(f'dataset/test/fresh/fresh_{i}.jpg', [0, 150, 0], "FRESH")
        create_sample_image(f'dataset/test/rotten/rotten_{i}.jpg', [150, 0, 0], "ROTTEN")

create_sample_dataset()

# ==============================
# 🚀 Data Loaders
# ==============================
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.1,
                                   height_shift_range=0.1, shear_range=0.1, zoom_range=0.1,
                                   horizontal_flip=True, fill_mode='nearest')
val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('dataset/train', target_size=IMG_SIZE,
                                                    batch_size=BATCH_SIZE, class_mode='categorical')
val_generator = val_test_datagen.flow_from_directory('dataset/validation', target_size=IMG_SIZE,
                                                     batch_size=BATCH_SIZE, class_mode='categorical')
test_generator = val_test_datagen.flow_from_directory('dataset/test', target_size=IMG_SIZE,
                                                      batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False)

# ==============================
# 🧠 Build Model
# ==============================
base_model = EfficientNetB0(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss='categorical_crossentropy', metrics=['accuracy'])

# ==============================
# 🏋 Train Model
# ==============================
callbacks = [EarlyStopping(patience=3, restore_best_weights=True),
             ModelCheckpoint("best_model.h5", save_best_only=True, monitor='val_accuracy')]

history = model.fit(train_generator, steps_per_epoch=train_generator.samples // BATCH_SIZE,
                    validation_data=val_generator,
                    validation_steps=val_generator.samples // BATCH_SIZE,
                    epochs=EPOCHS, callbacks=callbacks)

# ==============================
# 📊 Confusion Matrix
# ==============================
y_true = test_generator.classes
y_pred_probs = model.predict(test_generator)
y_pred = np.argmax(y_pred_probs, axis=1)
class_labels = list(train_generator.class_indices.keys())

cm = confusion_matrix(y_true, y_pred)
ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels).plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix")
plt.savefig("confusion_matrix.png")
plt.close()

# ==============================
# 🎯 Use Fixed Test Samples
# ==============================
fixed_test_images = [
    'dataset/test/fresh/fresh_0.jpg',
    'dataset/test/fresh/fresh_1.jpg',
    'dataset/test/rotten/rotten_0.jpg',
    'dataset/test/rotten/rotten_1.jpg',
]

def load_and_predict_fixed_images(image_paths):
    results = []
    for path in image_paths:
        img = Image.open(path).resize(IMG_SIZE)
        img_array = np.expand_dims(np.array(img) / 255.0, axis=0)
        prediction = model.predict(img_array)
        class_idx = np.argmax(prediction[0])
        confidence = float(np.max(prediction[0]))
        predicted_label = class_labels[class_idx]

        annotated = np.array(img).copy()
        annotated = cv2.putText(annotated, f"{predicted_label.upper()} ({confidence:.2f})", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        results.append(Image.fromarray(annotated))
    return results

sample_images = load_and_predict_fixed_images(fixed_test_images)

# ==============================
# 🖼 Gradio UI (No Upload)
# ==============================
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 Smart Fruit Freshness Classifier")

    gr.Markdown("### 📊 Confusion Matrix")
    gr.Image("confusion_matrix.png")

    gr.Markdown("### 🎯 Predictions on Fixed Test Samples")
    with gr.Row():
        for sample_img in sample_images:
            gr.Image(value=sample_img, show_label=False)

demo.launch()

Found 100 images belonging to 2 classes.
Found 30 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4023 - loss: 0.7478   

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3/3 ━━━━━━━━━━━━━━━━━━━━ 35s 5s/step - accuracy: 0.4047 - loss: 0.7447 - val_accuracy: 0.5000 - val_loss: 0.6970
Epoch 2/10
1/3 ━━━━━━━━━━━━━━━━━━━━ 7s 4s/step - accuracy: 0.5312 - loss: 0.7056

...

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.7500 - loss: 0.6307 - val_accuracy: 0.5000 - val_loss: 0.6997


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6bfa4d68cf0fd5da1b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
